In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Amyotrophic_Lateral_Sclerosis"
cohort = "GSE212131"

# Input paths
in_trait_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis"
in_cohort_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis/GSE212131"

# Output paths
out_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/GSE212131.csv"
out_gene_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE212131.csv"
out_clinical_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE212131.csv"
json_path = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Establishing mRNA and miRNA interactions driving disease heterogeneity in ALS patient survival (microarray)"
!Series_summary	"Transcriptomic analysis of lymphoblastoid cell lines from ALS patients with varying disease duration"
!Series_summary	"Amyotrophic lateral sclerosis (ALS) is a fatal neurodegenerative disease, associated with the degeneration of both upper and lower motor neurons of the motor cortex, brainstem and spinal cord. Death in most patients results from respiratory failure within 3-4 years from symptom onset. However, due to disease heterogeneity some individuals survive only months from symptom onset while others live for several years. Identifying specific biomarkers that aid in establishing disease prognosis, particularly in terms of predicting disease progression, will help our understanding of ALS pathophysiology and could be used to monitor a patient’s response to drugs and therapeutic agents.  Transcriptomic profiling techno

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from microarray analysis
is_gene_available = True  # Affymetrix Human Exon 1.0ST GeneChip microarrays were used

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait: The dataset is comparing ALS patients with short duration (<12 months) vs long duration (>6 years)
# This can be inferred from the background information, but not directly available in the sample characteristics
trait_row = None  # Not directly available in sample characteristics

# For age: Not provided in the sample characteristics
age_row = None

# For gender: Available in the sample characteristics at index 0
gender_row = 0

# 2.2 Data Type Conversion Functions

# For trait (not used since trait_row is None, but defined for completeness)
def convert_trait(value):
    return None  # Not directly available

# For age (not used since age_row is None, but defined for completeness)
def convert_age(value):
    return None

# For gender
def convert_gender(value):
    if value is None:
        return None
    # Extract the value after the colon
    if ":" in value:
        gender = value.split(":")[1].strip().lower()
        if "female" in gender:
            return 0
        elif "male" in gender:
            return 1
    return None

# 3. Save Metadata
# Trait data availability is determined by whether trait_row is None
is_trait_available = trait_row is not None

# Initial validation
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction - Skip this step since trait_row is None
# No clinical feature extraction needed as trait_row is None


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['2315554', '2315633', '2315674', '2315739', '2315894', '2315918',
       '2315951', '2316218', '2316245', '2316379', '2316558', '2316605',
       '2316746', '2316905', '2316953', '2317246', '2317317', '2317434',
       '2317472', '2317512'],
      dtype='object', name='ID')

Gene data dimensions: 22011 genes × 42 samples


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers in the data
# Looking at the identifiers '2315554', '2315633', etc., these appear to be probe IDs 
# from a microarray platform rather than standard human gene symbols.
# Standard human gene symbols would be alphanumeric like GAPDH, TP53, etc.
# These purely numeric identifiers likely need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // D

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns in the gene annotation dataframe
# From the preview, we can see:
# - 'ID' column contains the numeric identifiers (like 2315100) which match the gene expression indices
# - 'gene_assignment' column contains the gene symbols and additional information

# 2. Extract gene mapping from gene annotation
# We'll use the 'ID' and 'gene_assignment' columns for mapping
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')

# Print a preview of the gene mapping
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This handles many-to-many relationships between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the dimensions of the gene expression data after mapping
print(f"\nGene data dimensions after mapping: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Print the first 20 gene symbols after mapping
print("\nFirst 20 gene symbols after mapping:")
print(gene_data.index[:20])

# Check if any genes were filtered out during mapping (genes with no symbols)
print(f"\nNumber of genes after mapping: {len(gene_data)}")


Gene mapping preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'Gene': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771', '---', '---', '---', '---']}



Gene data dimensions after mapping: 48895 genes × 42 samples

First 20 gene symbols after mapping:
Index(['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1',
       'A1-', 'A10', 'A11', 'A12', 'A13', 'A14', 'A16', 'A1BG', 'A1BG-AS',
       'A1CF'],
      dtype='object', name='Gene')

Number of genes after mapping: 48895


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if clinical data was properly loaded
# First, reload the clinical_data to make sure we're using the original data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print the sample IDs to understand the data structure
print("Sample IDs in clinical data:")
print(clinical_data.columns[:5], "...")  # Show first 5 sample IDs

# Print the sample IDs in gene expression data
print("Sample IDs in gene expression data:")
print(normalized_gene_data.columns[:5], "...")  # Show first 5 sample IDs

# Explicitly set trait availability based on findings from previous steps
is_trait_available = False  # From step 2, we determined trait_row was None
linked_data = None

if is_trait_available:
    # This block won't execute because is_trait_available=False
    # But keeping it here for completeness
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=0,  # Placeholder, won't be used
        convert_trait=lambda x: None,  # Placeholder, won't be used
        age_row=None,
        convert_age=None,
        gender_row=0,
        convert_gender=lambda x: None  # Placeholder, won't be used
    )
    
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print(f"Clinical data preview: {preview_df(selected_clinical_df, n=3)}")
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    
    # 3. Handle missing values
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine if trait and demographic features are biased
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Data shape after removing biased features: {linked_data.shape}")
else:
    print("Trait data is unavailable in this dataset.")
    is_biased = True  # Dataset can't be used without trait data
    linked_data = pd.DataFrame()  # Empty DataFrame

# 5. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data if not linked_data.empty else pd.DataFrame(index=normalized_gene_data.columns),
    note="Dataset contains gene expression data from ALS patients, but lacks trait information (disease status) required for associational studies."
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associational studies.")

Gene data shape after normalization: (18418, 42)
First 5 gene symbols after normalization: Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE212131.csv
Sample IDs in clinical data:
Index(['!Sample_geo_accession', 'GSM6509811', 'GSM6509812', 'GSM6509813',
       'GSM6509814'],
      dtype='object') ...
Sample IDs in gene expression data:
Index(['GSM6509811', 'GSM6509812', 'GSM6509813', 'GSM6509814', 'GSM6509815'], dtype='object') ...
Trait data is unavailable in this dataset.
Abnormality detected in the cohort: GSE212131. Preprocessing failed.
Dataset deemed not usable for associational studies.
